In [1]:
# imports 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [51]:
# Assuming you have your data in a Pandas DataFrame called 'data'
# where the first 4 columns are your features and the last column is the target variable.

# reg_input_allconns == every single connectivity outputted by DCM (64) + anxiety / depression scores 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_allconns')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-2]  # all columns except 1st/last are features
y = data.iloc[:, -2]   # Last column is the target variable

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train the logistic regression model
logreg = LogisticRegression(random_state = 42)
logreg.fit(X_train, y_train)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg, X_train, y_train, cv=num_folds)
average_accuracy = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred = logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test,y_pred)

print("Prediction Accuracy using all connectivities + MH data:", accuracy_logreg)
print(f"\nAccuracy using {num_folds}-fold Cross-Validation: {average_accuracy:.2f}")


Prediction Accuracy using all connectivities + MH data: 0.8571428571428571

Accuracy using 10-fold Cross-Validation: 0.92


In [50]:
## feature selection -- reducing number of features to top importance 

# Step 4: Get the coefficients (feature importances) of the model
feature_importances = np.abs(logreg.coef_[0])  # Take the absolute values to handle negative coefficients

# Step 5: Select the top features with the highest importance/amount of covariance explained 
num_top_features = 4 # Change this number to select a different number of top features
top_feature_indices = np.argsort(feature_importances)[::-1][:num_top_features]
top_features = X.columns[top_feature_indices]

# Print the selected top features and covariance explained by feature 
print("Selected Top Features and Importance Scores:")
for feature, importance in zip(top_features, feature_importances[top_feature_indices]):
    print(f"{feature}: {importance}")

Selected Top Features and Importance Scores:
depression: 0.7728860160234696
anxiety: 0.6353272020887974
LeftAmy-LeftPul: 0.5182114308617533
RightPul-RightFFA: 0.43559666041015616


In [22]:
# Step 6: Retrain the logistic regression model using only the selected top features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

logreg_top_features = LogisticRegression()
logreg_top_features.fit(X_train_top, y_train)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg_top_features, X_train_top, y_train, cv=num_folds)
average_accuracy = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred_top = logreg_top_features.predict(X_test_top)
accuracy_logreg = accuracy_score(y_test,y_pred_top)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg_top_features.score(X_test_top, y_test)))
print(f"\nAccuracy using {num_folds}-fold Cross-Validation: {average_accuracy:.2f}")

Accuracy of logistic regression classifier on test set: 0.86

Accuracy using 10-fold Cross-Validation: 0.94


In [30]:
# testing model accuracy of just Mental Health data (anxiety / depression ratings )
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_mhonly')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-3]  # all columns except 1st/last are features
y = data.iloc[:, -3]   # Last column is the target variable

X_train_mh, X_test_mh, y_train_mh, y_test_mh = train_test_split(X, y, test_size=0.2, random_state=42)

logreg_mh = LogisticRegression()
logreg_mh.fit(X_train_mh, y_train_mh)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg_mh, X_train_mh, y_train_mh, cv=num_folds)
average_accuracy_mh = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred_mh = logreg_mh.predict(X_test_mh)
accuracy_logreg_mh = accuracy_score(y_test_mh,y_pred_mh)

## feature selection -- reducing number of features to top importance 

# Step 4: Get the coefficients (feature importances) of the model
feature_importances_mh = np.abs(logreg_mh.coef_[0])  # Take the absolute values to handle negative coefficients

# Step 5: Select the top features with the highest importance/amount of covariance explained 
num_top_features = 2 # Change this number to select a different number of top features
top_feature_indices_mh = np.argsort(feature_importances_mh)[::-1][:num_top_features]
top_features_mh = X.columns[top_feature_indices_mh]

# Print the selected top features and covariance explained by feature 
print("Covariance explained by each feature:")
for feature, importance in zip(top_features_mh, feature_importances_mh[top_feature_indices_mh]):
    print(f"{feature}: {importance}")

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg_mh.score(X_test_mh, y_test_mh)))
print(f"\nAccuracy using {num_folds}-fold Cross-Validation: {average_accuracy_mh:.2f}")

Covariance explained by each feature:
depression: 0.771328048639235
anxiety: 0.6256402425562101
Accuracy of logistic regression classifier on test set: 0.86

Accuracy using 10-fold Cross-Validation: 0.94


0     1
1     1
2     1
3     1
4     1
     ..
64    0
65    0
66    0
67    0
68    0
Name: group, Length: 69, dtype: int64

In [32]:
# testing model accuracy of mh data (anx/depression) + significantly different 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_limitedconns')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-2]  # all columns except 1st/last are features
y = data.iloc[:, -2]   # Last column is the target variable

X_train_mhconn2, X_test_mhconn2, y_train_mhconn2, y_test_mhconn2 = train_test_split(X, y, test_size=0.2, random_state=42)

logreg_mhconn2 = LogisticRegression()
logreg_mhconn2.fit(X_train_mhconn2, y_train_mhconn2)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg_mhconn2, X_train_mhconn2, y_train_mhconn2, cv=num_folds)
average_accuracy_mhconn2 = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred_mhconn2 = logreg_mhconn2.predict(X_test_mhconn2)
accuracy_logreg_mhconn2 = accuracy_score(y_test_mhconn2,y_pred_mhconn2)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg_mhconn2.score(X_test_mhconn2, y_test_mhconn2)))
print(f"\nAccuracy using {num_folds}-fold Cross-Validation: {average_accuracy_mhconn2:.2f}")


Accuracy of logistic regression classifier on test set: 0.86

Accuracy using 10-fold Cross-Validation: 0.94


0     1
1     1
2     1
3     1
4     1
     ..
64    0
65    0
66    0
67    0
68    0
Name: group, Length: 69, dtype: int64

# Method 2 -- recursive feature elimination w k-fold CV

In [190]:
# Recursive feature elimination with cross-validation 
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# reg_input_allconns == every single connectivity outputted by DCM (64) + anxiety / depression scores 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_allconns')

feature_names = X.columns.values

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-1]  # All but first / last column are features
y = data.iloc[:, -1]   # Last column is the target variable

min_features_to_select = 1  # Minimum number of features to consider
clf = LogisticRegression()
cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)
rfecv.fit(X, y)

print(f"Optimal number of features: {rfecv.n_features_}")
print(
    "Features selected by forward sequential selection: "
    f"{feature_names[rfecv.get_support()]}"
)


Optimal number of features: 2
Features selected by forward sequential selection: ['LeftAmy-LeftPul' 'depression']


In [192]:
# Recursive feature elimination with cross-validation -- mh only  
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# reg_input_allconns == every single connectivity outputted by DCM (64) + anxiety / depression scores 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_mhonly')


# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-1]  # All but first / last column are features
y = data.iloc[:, -1]   # Last column is the target variable
feature_names = X.columns.values

min_features_to_select = 1  # Minimum number of features to consider
clf = LogisticRegression()
cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)
rfecv.fit(X, y)

print(f"Optimal number of features: {rfecv.n_features_}")
print(
    "Features selected by forward sequential selection: "
    f"{feature_names[rfecv.get_support()]}"
)

Optimal number of features: 1
Features selected by forward sequential selection: ['depression']


In [188]:
# Recursive feature elimination with cross-validation -- mh + significantly diff connectivies   
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# reg_input_allconns == every single connectivity outputted by DCM (64) + anxiety / depression scores 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_limitedconns')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-1]  # All but first / last column are features
y = data.iloc[:, -1]   # Last column is the target variable

feature_names = X.columns.values

min_features_to_select = 1  # Minimum number of features to consider
clf = LogisticRegression()
cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)
rfecv.fit(X, y)

print(f"Optimal number of features: {rfecv.n_features_}")
print(
    "Features selected by forward sequential selection: "
    f"{feature_names[rfecv.get_support()]}"
)


Optimal number of features: 1
Features selected by forward sequential selection: ['depression']


# Method 3: sequential feature selection (both forward / backward)

In [211]:
## # sequential feature selector 

from sklearn.feature_selection import SequentialFeatureSelector
from time import time
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeCV

# reg_input_allconns == every single connectivity outputted by DCM (64) + anxiety / depression scores 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_allconns')
X = data.iloc[:, 1:-1]  # 2nd - 2nd to end columns are features (ignoring subjectname + group)
y = data.iloc[:, -1]   # Last column is the target variable

feature_names = X.columns.values

logreg1 = LogisticRegression()
#logreg_ridge = RidgeCV(logreg1.fit(X,y))
# starts with no features and adds one by one 
tic_fwd = time()
sfs_forward = SequentialFeatureSelector(
    logreg1, n_features_to_select=3, direction="forward"
).fit(X, y)
toc_fwd = time()

# backwards -- starts w all features and slowly adds each one 
tic_bwd = time()
sfs_backward = SequentialFeatureSelector(
    logreg1, n_features_to_select=3, direction="backward"
).fit(X, y)
toc_bwd = time()

print(
    "Features selected by forward sequential selection: "
    f"{feature_names[sfs_forward.get_support()]}"
)
print(f"Done in {toc_fwd - tic_fwd:.3f}s")
print(
    "Features selected by backward sequential selection: "
    f"{feature_names[sfs_backward.get_support()]}"
)
print(f"Done in {toc_bwd - tic_bwd:.3f}s")


Features selected by forward sequential selection: ['LeftAmy-LeftAmy' 'LeftAmy-LeftPul' 'depression']
Done in 1.527s
Features selected by backward sequential selection: ['RightPul-RightAmy' 'RightPul-RightPul' 'depression']
Done in 10.536s


# Logistic regression for mental health data (getting stats)
Averaged anxiety / depression scores since they were so correlated -- median = 4, marked all participants that were 4 or above as 1 (high dep / anxiety group) and all that were below 4 as 0 (low dep/anx group) 

want to see conn ability to predict high/low dep/anx 

In [40]:
# testing model accuracy of mh data (anx/depression) + significantly different connectivites 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_limitedconns')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-4]  # just connectivities 
y = data.iloc[:, -1]   # Last column is the target variable -- mh group 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg, X_train, y_train, cv=num_folds)
average_accuracy = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred = logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test,y_pred)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
print(f"\nAccuracy using {num_folds}-fold Cross-Validation: {average_accuracy:.2f}")


Accuracy of logistic regression classifier on test set: 0.29

Accuracy using 10-fold Cross-Validation: 0.61


In [43]:
import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LogisticRegression

data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_limitedconns')
X = data.iloc[:, 1:-4]  # just connectivities 
y = data.iloc[:, -1]   # Last column is the target variable -- mh group 

def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

model = LogisticRegression()
model.fit(X, y)
print(logit_pvalue(model, X))

# compare with statsmodels
import statsmodels.api as sm
sm_model = sm.Logit(y, sm.add_constant(X)).fit(disp=0)
print(sm_model.pvalues)
sm_model.summary()

[0.88785719 0.25673109 0.67272688]
const                0.416976
LeftAmy-LeftmFOC     0.025669
RightAmy-RightPul    0.170549
dtype: float64


/Users/gilliangrennan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               mh_group   No. Observations:                   69
Model:                          Logit   Df Residuals:                       66
Method:                           MLE   Df Model:                            2
Date:                Mon, 31 Jul 2023   Pseudo R-squ.:                 0.07758
Time:                        12:30:06   Log-Likelihood:                -43.950
converged:                       True   LL-Null:                       -47.646
Covariance Type:            nonrobust   LLR p-value:                   0.02482
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.2563      0.316     -0.812      0.417      -0.875       0.363
LeftAmy-LeftmFOC      2.5942      1.163      2.231      0.026       0.315       4.873
RightAmy-RightPul    -2.2215      1.621     -1.370      0.171      -5.399       0.956
=====================================================================================
"""

In [47]:
# testing model accuracy of mh data (anx/depression) + significantly different connectivites 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_allconns')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-4]  # just connectivities 
y = data.iloc[:, -1]   # Last column is the target variable -- mh group 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg, X_train, y_train, cv=num_folds)
average_accuracy = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred = logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test,y_pred)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
print(f"\nAccuracy using {num_folds}-fold Cross-Validation: {average_accuracy:.2f}")

Accuracy of logistic regression classifier on test set: 0.21

Accuracy using 10-fold Cross-Validation: 0.66


In [48]:
model = LogisticRegression()
model.fit(X, y)
print(logit_pvalue(model, X))

# compare with statsmodels
import statsmodels.api as sm
sm_model = sm.Logit(y, sm.add_constant(X)).fit(disp=0)
print(sm_model.pvalues)
sm_model.summary()

[0.97492716 0.96638671 0.60531765 0.50912767 0.70797272 0.95523025
 0.83401453 0.98315302 0.81286307 0.78408757 0.92619781 0.97789254
 0.87503909 0.97971589 0.92440409 0.92665707 0.98189847 0.91703737
 0.92750981 0.90437824 0.85565802 0.95747752 0.83096205 0.77966446
 0.99237437]
const                  0.627265
LeftAmy-LeftAmy        0.941911
LeftAmy-LeftFFA        0.147792
LeftAmy-LeftmFOC       0.037179
LeftAmy-LeftPul        0.009778
RightAmy-RightAmy      0.542987
RightAmy-RightFFA      0.976901
RightAmy-RightmFOC     0.846783
RightAmy-RightPul      0.420621
LeftFFA-LeftAmy        0.983108
LeftFFA-LeftFFA        0.688643
LeftFFA-LeftPul        0.702738
RightFFA-RightAmy      0.558136
RightFFA-RightFFA      0.956026
RightFFA-RightPul      0.844089
LeftmOFC-LeftAmy       0.483366
LeftmOFC-LeftmOFC      0.715417
RightmOFC-RightAmy     0.957264
RightmOFC-RightmOFC    0.059791
LeftPul-LeftAmy        0.968507
LeftPul-LeftFFA        0.365575
LeftPul-LeftPul        0.195653
RightPul-RightA

/Users/gilliangrennan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               mh_group   No. Observations:                   69
Model:                          Logit   Df Residuals:                       44
Method:                           MLE   Df Model:                           24
Date:                Mon, 31 Jul 2023   Pseudo R-squ.:                  0.3223
Time:                        12:32:48   Log-Likelihood:                -32.289
converged:                       True   LL-Null:                       -47.646
Covariance Type:            nonrobust   LLR p-value:                    0.1622
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.5177      1.066      0.486      0.627      -1.572       2.607
LeftAmy-LeftAmy        -0.5851      8.030     -0.073      0.942     -16.323      15.153
LeftAmy-LeftFFA        -3.5033      2.420     -1.447      0.148      -8.247       1.241
LeftAmy-LeftmFOC        5.9262      2.844      2.084      0.037       0.352      11.500
LeftAmy-LeftPul        -7.2637      2.811     -2.584      0.010     -12.774      -1.753
RightAmy-RightAmy       3.5532      5.841      0.608      0.543      -7.895      15.002
RightAmy-RightFFA      -0.0593      2.047     -0.029      0.977      -4.072       3.953
RightAmy-RightmFOC     -0.4080      2.111     -0.193      0.847      -4.546       3.730
RightAmy-RightPul      -2.6151      3.247     -0.805      0.421      -8.980       3.749
LeftFFA-LeftAmy         0.0426      2.014      0.021      0.983      -3.904       3.990
LeftFFA-LeftFFA        -1.7642      4.403     -0.401      0.689     -10.393       6.865
LeftFFA-LeftPul        -0.9789      2.565     -0.382      0.703      -6.006       4.048
RightFFA-RightAmy      -1.6141      2.756     -0.586      0.558      -7.016       3.788
RightFFA-RightFFA      -0.2472      4.483     -0.055      0.956      -9.033       8.539
RightFFA-RightPul       0.4667      2.373      0.197      0.844      -4.184       5.118
LeftmOFC-LeftAmy        3.8439      5.484      0.701      0.483      -6.905      14.593
LeftmOFC-LeftmOFC      -3.0707      8.422     -0.365      0.715     -19.578      13.437
RightmOFC-RightAmy     -0.2505      4.674     -0.054      0.957      -9.411       8.910
RightmOFC-RightmOFC    13.6106      7.231      1.882      0.060      -0.561      27.783
LeftPul-LeftAmy         0.1036      2.624      0.039      0.969      -5.039       5.246
LeftPul-LeftFFA        -1.4670      1.621     -0.905      0.366      -4.645       1.711
LeftPul-LeftPul         9.1595      7.078      1.294      0.196      -4.714      23.033
RightPul-RightAmy      -3.1554      2.562     -1.232      0.218      -8.177       1.866
RightPul-RightFFA       1.4246      2.190      0.650      0.515      -2.869       5.718
RightPul-RightPul      -2.5334      6.764     -0.375      0.708     -15.790      10.723
=======================================================================================
"""

# Just using connectivities (no mental health data included as features)

In [58]:
# Assuming you have your data in a Pandas DataFrame called 'data'
# where the first 4 columns are your features and the last column is the target variable.

# reg_input_allconns == every single connectivity outputted by DCM (64) + anxiety / depression scores 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_allconns')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-4]  # all columns except 1st/last are features
y = data.iloc[:, -2]   # Last column is the target variable

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train the logistic regression model
logreg = LogisticRegression(random_state = 42)
logreg.fit(X_train, y_train)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg, X_train, y_train, cv=num_folds)
average_accuracy = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred = logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test,y_pred)

print("Prediction Accuracy using all connectivities + MH data:", accuracy_logreg)
print(f"\nAccuracy using {num_folds}-fold Cross-Validation: {average_accuracy:.2f}")

Prediction Accuracy using all connectivities + MH data: 0.42857142857142855

Accuracy using 10-fold Cross-Validation: 0.58


In [59]:
# testing model accuracy of mh data (anx/depression) + significantly different 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_limitedconns')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-4]  # all columns except 1st/last are features
y = data.iloc[:, -2]   # Last column is the target variable

X_train_mhconn2, X_test_mhconn2, y_train_mhconn2, y_test_mhconn2 = train_test_split(X, y, test_size=0.2, random_state=42)

logreg_mhconn2 = LogisticRegression()
logreg_mhconn2.fit(X_train_mhconn2, y_train_mhconn2)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg_mhconn2, X_train_mhconn2, y_train_mhconn2, cv=num_folds)
average_accuracy_mhconn2 = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred_mhconn2 = logreg_mhconn2.predict(X_test_mhconn2)
accuracy_logreg_mhconn2 = accuracy_score(y_test_mhconn2,y_pred_mhconn2)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg_mhconn2.score(X_test_mhconn2, y_test_mhconn2)))
print(f"\nAccuracy using {num_folds}-fold Cross-Validation: {average_accuracy_mhconn2:.2f}")

Accuracy of logistic regression classifier on test set: 0.43

Accuracy using 10-fold Cross-Validation: 0.67


In [62]:
# Recursive feature elimination with cross-validation 
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

# reg_input_allconns == every single connectivity outputted by DCM (64) + anxiety / depression scores 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_allconns')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-4]  # All but first / last column are features
y = data.iloc[:, -2]   # Last column is the target variable
feature_names = X.columns.values

min_features_to_select = 1  # Minimum number of features to consider
clf = LogisticRegression()
cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)
rfecv.fit(X, y)

print(f"Optimal number of features: {rfecv.n_features_}")
print(
    "Features selected by forward sequential selection: "
    f"{feature_names[rfecv.get_support()]}"
)

Optimal number of features: 8
Features selected by forward sequential selection: ['LeftAmy-LeftFFA' 'LeftAmy-LeftmFOC' 'RightAmy-RightPul'
 'LeftFFA-LeftAmy' 'LeftFFA-LeftFFA' 'RightmOFC-RightAmy'
 'LeftPul-LeftPul' 'RightPul-RightAmy']


In [97]:
# Finding top 2 features 

# reg_input_allconns == every single connectivity outputted by DCM (64) + anxiety / depression scores 
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_allconns')

# Step 1: Import data and separate features (X) and target (y)
X = data.iloc[:, 1:-4]  # all columns except 1st/last are features
y = data.iloc[:, -2]   # Last column is the target variable

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Train the logistic regression model
logreg = LogisticRegression(random_state = 42)
logreg.fit(X_train, y_train)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg, X_train, y_train, cv=num_folds)
average_accuracy = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred = logreg.predict(X_test)
accuracy_logreg = accuracy_score(y_test,y_pred)

print(f"\nAccuracy using {num_folds}-fold Cross-Validation using all 26 connectivities: {average_accuracy:.2f}")
## feature selection -- reducing number of features to top importance 

# Step 4: Get the coefficients (feature importances) of the model
feature_importances = np.abs(logreg.coef_[0])  # Take the absolute values to handle negative coefficients

# Step 5: Select the top features with the highest importance/amount of covariance explained 
num_top_features = 1 # Change this number to select a different number of top features
top_feature_indices = np.argsort(feature_importances)[::-1][:num_top_features]
top_features = X.columns[top_feature_indices]

# Print the selected top features and covariance explained by feature 
print(f"\nSelected Top Features and Importance Scores:")
for feature, importance in zip(top_features, feature_importances[top_feature_indices]):
    print(f"{feature}: {importance}")
    
# Step 6: Retrain the logistic regression model using only the selected top features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

logreg_top_features = LogisticRegression()
logreg_top_features.fit(X_train_top, y_train)

# Evaluate the model performance using k-fold cross-validation
num_folds = 10  # Change this number to modify the number of folds
cv_accuracy = cross_val_score(logreg_top_features, X_train_top, y_train, cv=num_folds)
average_accuracy = np.mean(cv_accuracy)

# Evaluate prediction accuracy 
y_pred_top = logreg_top_features.predict(X_test_top)
accuracy_logreg = accuracy_score(y_test,y_pred_top)

print(f"\nAccuracy using {num_folds}-fold Cross-Validation just using top 2 features: {average_accuracy:.2f}")

X_top = X[top_features]
logreg_top_features = LogisticRegression()
logreg_top_features.fit(X_top, y)

def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

print(logit_pvalue(logreg_top_features, X_top))

# compare with statsmodels
import statsmodels.api as sm
sm_model = sm.Logit(y, sm.add_constant(X_top)).fit(disp=0)
print(sm_model.pvalues)
sm_model.summary()


Accuracy using 10-fold Cross-Validation using all 26 connectivities: 0.58

Selected Top Features and Importance Scores:
LeftAmy-LeftmFOC: 0.9634015004586973

Accuracy using 10-fold Cross-Validation just using top 2 features: 0.60
[0.26576606 0.27816265]
const               0.083143
LeftAmy-LeftmFOC    0.037023
dtype: float64


/Users/gilliangrennan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  group   No. Observations:                   69
Model:                          Logit   Df Residuals:                       67
Method:                           MLE   Df Model:                            1
Date:                Mon, 31 Jul 2023   Pseudo R-squ.:                 0.05219
Time:                        13:38:32   Log-Likelihood:                -45.159
converged:                       True   LL-Null:                       -47.646
Covariance Type:            nonrobust   LLR p-value:                   0.02574
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.5621      0.324     -1.733      0.083      -1.198       0.074
LeftAmy-LeftmFOC     2.3504      1.127      2.086      0.037       0.141       4.559
====================================================================================
"""

In [69]:
data = pd.read_excel('data.xlsx',sheet_name = 'reg_input_allconns')
X = data.iloc[:, 1:-4]  # just connectivities 
y = data.iloc[:, -2]   # Last column is the target variable -- mh group 

def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

model = LogisticRegression()
model.fit(X, y)
print(logit_pvalue(model, X))

# compare with statsmodels
import statsmodels.api as sm
sm_model = sm.Logit(y, sm.add_constant(X)).fit(disp=0)
print(sm_model.pvalues)
sm_model.summary()

[0.27813239 0.27353473 0.50524946]
const                0.079426
LeftAmy-LeftmFOC     0.026012
RightAmy-RightPul    0.038401
dtype: float64


/Users/gilliangrennan/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  group   No. Observations:                   69
Model:                          Logit   Df Residuals:                       66
Method:                           MLE   Df Model:                            2
Date:                Mon, 31 Jul 2023   Pseudo R-squ.:                  0.1083
Time:                        13:01:52   Log-Likelihood:                -42.483
converged:                       True   LL-Null:                       -47.646
Covariance Type:            nonrobust   LLR p-value:                  0.005728
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.5874      0.335     -1.754      0.079      -1.244       0.069
LeftAmy-LeftmFOC      2.6658      1.198      2.226      0.026       0.319       5.013
RightAmy-RightPul    -3.9720      1.918     -2.071      0.038      -7.732      -0.212
=====================================================================================
"""